# Task 1.2: Geolocation Analysis - IP to Country Mapping

## Objective
Enrich transaction data with geographic information by:
1. Converting IP addresses to integer format
2. Mapping IP addresses to countries using range-based lookup
3. Analyzing fraud patterns by country

## Why Geolocation Matters for Fraud Detection
- Transactions from certain regions may have higher fraud rates
- Mismatches between user location and transaction location can signal fraud
- Geographic velocity (transactions from distant locations in short time) indicates fraud

In [ ]:
# Standard imports
import sys
from pathlib import Path

# Add project root to path for imports
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Project imports
from src.data.loader import load_fraud_data, load_ip_country_data
from src.data.geo import ip_to_int, ip_series_to_int, merge_ip_to_country, get_country_fraud_stats

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 1. Load Data

In [ ]:
# Load cleaned fraud data from previous notebook
CLEANED_DATA_PATH = project_root / "data" / "processed" / "fraud_cleaned.parquet"

# If cleaned data exists, use it; otherwise load raw and clean
if CLEANED_DATA_PATH.exists():
    df_fraud = pd.read_parquet(CLEANED_DATA_PATH)
    print(f"Loaded cleaned data: {df_fraud.shape}")
else:
    from src.data.cleaning import clean_fraud_data
    df_raw = load_fraud_data(project_root / "data" / "raw" / "Fraud_Data.csv")
    df_fraud, _ = clean_fraud_data(df_raw)
    print(f"Loaded and cleaned raw data: {df_fraud.shape}")

In [ ]:
# Load IP to country mapping
IP_COUNTRY_PATH = project_root / "data" / "raw" / "IpAddress_to_Country.csv"

df_ip_country = load_ip_country_data(IP_COUNTRY_PATH)
print(f"IP Country mapping shape: {df_ip_country.shape}")
df_ip_country.head()

## 2. Understand IP Address Format

In [ ]:
# Check the format of IP addresses in fraud data
print("IP Address column info:")
print(f"  Data type: {df_fraud['ip_address'].dtype}")
print(f"  Sample values:")
print(df_fraud['ip_address'].head(10))

In [ ]:
# Check the format of IP ranges in mapping data
print("\nIP Range column info:")
print(f"  Lower bound dtype: {df_ip_country['lower_bound_ip_address'].dtype}")
print(f"  Upper bound dtype: {df_ip_country['upper_bound_ip_address'].dtype}")
print(f"\nSample IP ranges:")
df_ip_country.head()

In [ ]:
# Check unique countries
print(f"\nNumber of unique countries: {df_ip_country['country'].nunique()}")
print(f"\nTop 10 countries by IP range count:")
print(df_ip_country['country'].value_counts().head(10))

### Interpretation: IP Address Format

*TODO: After running, describe:*
- What format are the IP addresses in? (numeric or dotted string)
- Are IP ranges already numeric or need conversion?
- How many countries are represented?

## 3. IP Address Conversion

In [ ]:
# Demonstrate IP to integer conversion
print("IP Conversion Examples:")
test_ips = ["192.168.1.1", "10.0.0.1", "255.255.255.255", 3232235777.0]
for ip in test_ips:
    result = ip_to_int(ip)
    print(f"  {ip} -> {result}")

In [ ]:
# Convert IP addresses in fraud data to integers
df_fraud['ip_int'] = ip_series_to_int(df_fraud['ip_address'])

print("IP conversion results:")
print(f"  Original column: ip_address")
print(f"  New column: ip_int")
print(f"  Invalid IPs (=-1): {(df_fraud['ip_int'] == -1).sum()}")

df_fraud[['ip_address', 'ip_int']].head(10)

## 4. Merge IP to Country

In [ ]:
# Perform the range-based merge to add country
df_with_country = merge_ip_to_country(
    df_fraud.drop(columns=['ip_int']),  # Remove temp column, function will recreate
    df_ip_country,
    ip_column='ip_address'
)

print(f"Merged data shape: {df_with_country.shape}")
print(f"New 'country' column added: {'country' in df_with_country.columns}")

In [ ]:
# Check merge results
country_counts = df_with_country['country'].value_counts()
unknown_count = (df_with_country['country'] == 'Unknown').sum()

print(f"Country mapping results:")
print(f"  Total transactions: {len(df_with_country):,}")
print(f"  Mapped to country: {len(df_with_country) - unknown_count:,}")
print(f"  Unknown/Unmapped: {unknown_count:,} ({unknown_count/len(df_with_country)*100:.2f}%)")
print(f"  Unique countries: {df_with_country['country'].nunique()}")

In [ ]:
# Sample of merged data
df_with_country[['ip_address', 'country', 'class']].head(10)

### Interpretation: IP to Country Merge

*TODO: After running, describe:*
- What percentage of IPs were successfully mapped?
- How many remained Unknown?
- Is the Unknown rate acceptable for analysis?

## 5. Fraud Analysis by Country

In [ ]:
# Calculate fraud statistics by country
country_stats = get_country_fraud_stats(df_with_country, target_col='class')

print("Top 15 Countries by Fraud Count:")
display(country_stats.head(15))

In [ ]:
# Countries with highest fraud rates (with minimum transaction count)
MIN_TRANSACTIONS = 100  # Filter out countries with too few transactions

country_stats_filtered = country_stats[country_stats['total_transactions'] >= MIN_TRANSACTIONS]
top_fraud_rate = country_stats_filtered.sort_values('fraud_rate', ascending=False).head(15)

print(f"\nTop 15 Countries by Fraud Rate (min {MIN_TRANSACTIONS} transactions):")
display(top_fraud_rate)

In [ ]:
# Visualize fraud by country
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Top countries by fraud count
top_by_count = country_stats.head(10)
axes[0].barh(top_by_count['country'], top_by_count['fraud_count'], color='#e74c3c')
axes[0].set_xlabel('Fraud Count')
axes[0].set_ylabel('Country')
axes[0].set_title('Top 10 Countries by Fraud Count')
axes[0].invert_yaxis()

# Top countries by fraud rate (filtered)
top_by_rate = country_stats_filtered.sort_values('fraud_rate', ascending=False).head(10)
axes[1].barh(top_by_rate['country'], top_by_rate['fraud_rate'] * 100, color='#c0392b')
axes[1].set_xlabel('Fraud Rate (%)')
axes[1].set_ylabel('Country')
axes[1].set_title(f'Top 10 Countries by Fraud Rate (min {MIN_TRANSACTIONS} txns)')
axes[1].invert_yaxis()

# Add overall fraud rate line
overall_rate = df_with_country['class'].mean() * 100
axes[1].axvline(x=overall_rate, color='black', linestyle='--', label=f'Overall: {overall_rate:.2f}%')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Transaction distribution by country (top 20)
top_countries = country_stats.head(20)

fig, ax = plt.subplots(figsize=(12, 6))

x = range(len(top_countries))
width = 0.35

non_fraud = top_countries['total_transactions'] - top_countries['fraud_count']
bars1 = ax.bar(x, non_fraud, width, label='Non-Fraud', color='#2ecc71')
bars2 = ax.bar(x, top_countries['fraud_count'], width, bottom=non_fraud, label='Fraud', color='#e74c3c')

ax.set_xlabel('Country')
ax.set_ylabel('Transaction Count')
ax.set_title('Transaction Distribution by Country (Top 20)')
ax.set_xticks(x)
ax.set_xticklabels(top_countries['country'], rotation=45, ha='right')
ax.legend()

plt.tight_layout()
plt.show()

### Interpretation: Geographic Fraud Patterns

*TODO: After running, describe:*
- Which countries have the highest absolute fraud counts?
- Which countries have the highest fraud rates?
- Are there countries with suspiciously high fraud rates?
- What business actions might this suggest? (e.g., extra verification for certain regions)

## 6. Summary and Next Steps

*TODO: Fill in after completing the analysis*

### Key Findings
1. **IP Mapping Success Rate**: [Describe percentage mapped]
2. **High-Risk Countries**: [List top countries by fraud rate]
3. **Geographic Patterns**: [Key observations]

### Business Implications
- [Recommendation 1]
- [Recommendation 2]

### Next Steps
- Proceed to feature engineering (time features, velocity features)
- Include country as a feature in the model

In [ ]:
# Save data with country for next notebook
output_path = project_root / "data" / "processed" / "fraud_with_country.parquet"
df_with_country.to_parquet(output_path, index=False)
print(f"Data with country saved to: {output_path}")